In [1]:

from ib_insync import *
util.startLoop()

In [26]:
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=22)

<IB connected to 127.0.0.1:7497 clientId=22>

In [27]:
#request all ticker at once in the beginning

def GetCurrentPrice(contract):
    temp_contract= contract
    ib.qualifyContracts(temp_contract)
    [ticker] = ib.reqTickers(temp_contract)
    ib.sleep(1)
    return ticker.marketPrice()
def GetWeightedVega(tte, vega):
    return vega
    
    
def AddGreeks(contract, position, avg_cost):
    greeks_dict = {}
    greeks_dict['symbol'] = contract.symbol
    
    if contract.secType != 'OPT':
        current_price = GetCurrentPrice(contract)
        greeks_dict['delta'] = position
        greeks_dict['cash_delta'] = position * current_price
        greeks_dict['expiry'] = dt.date(1900,1,1)
        greeks_dict['strike'] = 0
        greeks_dict['kind'] = "spot"
        greeks_dict['vega'] = 0
        greeks_dict['gamma'] = 0
        greeks_dict['theta'] = 0
        greeks_dict['weighted_vega'] = 0
        greeks_dict['cash_gamma'] = 0
        greeks_dict['avg_cost'] = avg_cost
        greeks_dict['current_price'] = current_price
    else:
        [ticker] = ib.reqTickers(contract)
        ib.sleep(1)
        print(ticker)
        greeks_dict['delta'] = ticker.modelGreeks.delta
#         print(type(position), type(ticker.modelGreeks.undPrice) , float(contract.multiplier))
        greeks_dict['cash_delta'] = position * ticker.modelGreeks.delta * ticker.modelGreeks.undPrice * float(contract.multiplier)
        greeks_dict['expiry'] = dt.datetime.strptime(contract.lastTradeDateOrContractMonth , '%Y%m%d').date()
        greeks_dict['strike'] = contract.strike
        greeks_dict['kind'] = contract.right
        greeks_dict['vega'] = ticker.modelGreeks.vega * np.sign(position)
        greeks_dict['gamma'] = ticker.modelGreeks.gamma
        greeks_dict['theta'] = ticker.modelGreeks.theta
        greeks_dict['weighted_vega'] = GetWeightedVega(ticker.modelGreeks.vega, 1)
        greeks_dict['cash_gamma'] = position * ticker.modelGreeks.gamma * ticker.modelGreeks.undPrice * float(contract.multiplier)
        greeks_dict['avg_cost'] = avg_cost
        greeks_dict['current_price'] = ticker.modelGreeks.optPrice
#     print(greeks_dict)
    return greeks_dict


### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [32]:
all_positions = ib.positions()
all_positions_dict_list = []

In [33]:
amc_positions = [x for x in all_positions if x.contract.symbol== 'GME']


In [34]:
# amc_positions

In [38]:
list_of_pos_dict = []
for position in amc_positions:
    contract = position.contract
    contract.exchange = 'SMART'
    position_dict = AddGreeks(contract, position.position, position.avgCost )
    list_of_pos_dict.append(position_dict)
    


Ticker(contract=Option(conId=495098693, symbol='GME', lastTradeDateOrContractMonth='20210611', strike=580.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='GME   210611C00580000', tradingClass='GME'), time=datetime.datetime(2021, 6, 10, 13, 49, 33, 204575, tzinfo=datetime.timezone.utc), bid=0.22, bidSize=74, ask=0.23, askSize=6, last=0.22, lastSize=1, prevBid=0.29, prevBidSize=19, prevAsk=0.3, prevAskSize=79, prevLast=0.29, prevLastSize=12, volume=852, high=0.65, low=0.2, close=1.47, halted=0.0, bidGreeks=OptionComputation(impliedVol=4.484728544143679, delta=0.0032226983544991795, optPrice=0.2199999988079071, pvDividend=0.0, gamma=0.000134312865148688, vega=0.001305840125614341, theta=-0.2199999988079071, undPrice=275.3443908691406), askGreeks=OptionComputation(impliedVol=4.5072680056757095, delta=0.00334237473527889, optPrice=0.23000000417232513, pvDividend=0.0, gamma=0.00013822674735654647, vega=0.0013057510217148877, theta=-0.23000000417232513, undPrice=

In [39]:
df = pd.DataFrame(list_of_pos_dict)

In [40]:
df

,symbol,delta,cash_delta,expiry,strike,kind,vega,gamma,theta,weighted_vega,cash_gamma,avg_cost,current_price
0,GME,0.003327,-824.546435,2021-06-11,580.0,C,-0.001306,0.000138,-0.069180,1,-34.132721,524.542522,0.069180
1,GME,-0.652313,-17907.288570,2021-06-11,290.0,P,0.058348,0.011134,-4.908713,1,305.655446,2750.757800,22.775731
2,GME,-0.744638,102015.410614,2021-06-11,300.0,P,-0.052570,0.009345,-4.410756,1,-1280.209159,2720.838320,30.987230
3,GME,-0.550582,-75349.867296,2021-06-11,280.0,P,0.064551,0.011983,-5.260013,1,1639.924658,1844.145800,16.761824


In [41]:
df.sum()

symbol           GMEGMEGMEGME
delta                -1.94421
cash_delta            7933.71
strike                   1450
kind                     CPPP
vega                 0.069023
gamma               0.0325994
theta                -14.6487
weighted_vega               4
cash_gamma            631.238
avg_cost              7840.28
current_price          70.594
dtype: object